This file allows for testing the GraphWorld setup with GNN implementations.
It is currently set up to test the SSL methods for the JL benchmarker.

Through this notebook you can attach a debugger.
Note that graph_tool does not work on windows, so we cannot use the graph generators.
Instead, we use the standard datasets from PyG.

In [1]:
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

from graph_world.self_supervised_learning.benchmarker_jl import NNNodeBenchmarkerJL
from graph_world.models.basic_gnn import GCN
from torch_geometric.datasets import Planetoid

from graph_world.self_supervised_learning.pretext_tasks.auxiliary_property_based import *

In [2]:
# Parameter setup (for cora)
benchmark_params = {
    'epochs' : 50,
    'lr' : 0.01,
    'lambda' : 1
}

h_params = {
    'in_channels' : 1433,
    'hidden_channels' : 16,
    'num_layers' : 2,
    'dropout' : 0.5,
    "embedding_corruption_ratio" : 0.1, 
    "partial_embedding_reconstruction" : True,
    'n_parts': 10,
    'shortest_path_cutoff': 6,
    'N_classes': 4,
    'k_largest': 20
}

generator_config = {
    'num_clusters' : 7,
}

pretext_tasks = [PairwiseAttrSim]

In [3]:
# Get dataset
dataset = Planetoid(root='/tmp/Cora', name='Cora')[0]
# dataset = KarateClub()[0]


In [4]:
import torch
def get_top_k_indices(input: torch.Tensor, k : int, largest : bool = True):
    '''
    Get the indices of the top K elements.

    Params
    ------
    input:
        The tensor to find the top k elements
    k:
        The number of top elements to find
    largest:
        Whether to find the largest or smallest

    Returns
    -------
    (row_indices, col_indices) where input[row_indices, col_indices] returns the top k elements.

    '''
    assert input.dim() == 2
    N_cols = input.shape[1]
    
    top_k_indices = input.view(-1).topk(k=k, largest=largest).indices
    
    row = torch.div(top_k_indices, N_cols, rounding_mode='floor')
    col = top_k_indices % N_cols
    return row, col

In [5]:
from sklearn.metrics.pairwise import cosine_similarity

A = torch.tensor([
    [1, 2, 3],
    [-1, 2, 3],
    [-5, -4, -3]
])



In [6]:
# Training. You can attach a debugger to w/e is needed inside train
benchmarker = NNNodeBenchmarkerJL(generator_config=generator_config, model_class=GCN, 
                benchmark_params=benchmark_params, h_params=h_params, pretext_tasks=pretext_tasks)
# benchmarker.SetMasks(train_mask=dataset.train_mask, val_mask=~dataset.train_mask, test_mask=~dataset.train_mask)
benchmarker.SetMasks(train_mask=dataset.train_mask, val_mask=dataset.val_mask, test_mask=dataset.test_mask)
benchmarker.train(data=dataset, tuning_metric="rocauc_ovr", tuning_metric_is_loss=False)

GCN(1433, 16, num_layers=2)


([2.563347578048706,
  2.530860424041748,
  2.490342617034912,
  2.4498486518859863,
  2.420605182647705,
  2.3336455821990967,
  2.1527934074401855,
  2.1563191413879395,
  2.2057878971099854,
  1.9645298719406128,
  1.9243029356002808,
  2.1821017265319824,
  1.8355568647384644,
  1.8805077075958252,
  1.8921842575073242,
  1.7479959726333618,
  1.6475868225097656,
  1.727056622505188,
  1.5740995407104492,
  1.5220037698745728,
  1.4906378984451294,
  1.4472354650497437,
  1.424708366394043,
  1.2783381938934326,
  1.2113617658615112,
  1.23779296875,
  1.1917037963867188,
  1.1693215370178223,
  1.1046885251998901,
  1.0020866394042969,
  1.1463239192962646,
  1.0595877170562744,
  0.9388185143470764,
  0.8834372758865356,
  1.0160521268844604,
  0.8465023636817932,
  0.9598731994628906,
  0.8512958288192749,
  0.9035521745681763,
  0.7628231048583984,
  0.7518360614776611,
  0.7520517110824585,
  0.777840793132782,
  0.8024308681488037,
  0.6934787034988403,
  0.6656278371810913,
